In [2]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### URL's

In [6]:
metadata = {
    "odata.metadata":"https://opendata.cbs.nl/ODataApi/OData/7233/$metadata", "value":[
        {
            "name":"TableInfos","url":"https://opendata.cbs.nl/ODataApi/odata/7233/TableInfos"
        },
        {
            "name":"UntypedDataSet","url":"https://opendata.cbs.nl/ODataApi/odata/7233/UntypedDataSet"
        },
        {
            "name":"TypedDataSet","url":"https://opendata.cbs.nl/ODataApi/odata/7233/TypedDataSet"
        },
        {
            "name":"DataProperties","url":"https://opendata.cbs.nl/ODataApi/odata/7233/DataProperties"
        },
        {
            "name":"CategoryGroups","url":"https://opendata.cbs.nl/ODataApi/odata/7233/CategoryGroups"
        },
        {
            "name":"Geslacht","url":"https://opendata.cbs.nl/ODataApi/odata/7233/Geslacht"
        },
        {
            "name":"Leeftijd","url":"https://opendata.cbs.nl/ODataApi/odata/7233/Leeftijd"
        },
        {
            "name":"Perioden","url":"https://opendata.cbs.nl/ODataApi/odata/7233/Perioden"
        }
    ]
}

In [6]:
response = requests.get('https://opendata.cbs.nl/ODataApi/odata/7052_95')
metadata = response.json()
metadata

{'odata.metadata': 'https://opendata.cbs.nl/ODataApi/OData/7052_95/$metadata',
 'value': [{'name': 'TableInfos',
   'url': 'https://opendata.cbs.nl/ODataApi/odata/7052_95/TableInfos'},
  {'name': 'UntypedDataSet',
   'url': 'https://opendata.cbs.nl/ODataApi/odata/7052_95/UntypedDataSet'},
  {'name': 'TypedDataSet',
   'url': 'https://opendata.cbs.nl/ODataApi/odata/7052_95/TypedDataSet'},
  {'name': 'DataProperties',
   'url': 'https://opendata.cbs.nl/ODataApi/odata/7052_95/DataProperties'},
  {'name': 'CategoryGroups',
   'url': 'https://opendata.cbs.nl/ODataApi/odata/7052_95/CategoryGroups'},
  {'name': 'Geslacht',
   'url': 'https://opendata.cbs.nl/ODataApi/odata/7052_95/Geslacht'},
  {'name': 'Leeftijd',
   'url': 'https://opendata.cbs.nl/ODataApi/odata/7052_95/Leeftijd'},
  {'name': 'Perioden',
   'url': 'https://opendata.cbs.nl/ODataApi/odata/7052_95/Perioden'}]}

In [7]:
urls = {d['name']: d['url'] for d in metadata['value']}
urls

{'TableInfos': 'https://opendata.cbs.nl/ODataApi/odata/7052_95/TableInfos',
 'UntypedDataSet': 'https://opendata.cbs.nl/ODataApi/odata/7052_95/UntypedDataSet',
 'TypedDataSet': 'https://opendata.cbs.nl/ODataApi/odata/7052_95/TypedDataSet',
 'DataProperties': 'https://opendata.cbs.nl/ODataApi/odata/7052_95/DataProperties',
 'CategoryGroups': 'https://opendata.cbs.nl/ODataApi/odata/7052_95/CategoryGroups',
 'Geslacht': 'https://opendata.cbs.nl/ODataApi/odata/7052_95/Geslacht',
 'Leeftijd': 'https://opendata.cbs.nl/ODataApi/odata/7052_95/Leeftijd',
 'Perioden': 'https://opendata.cbs.nl/ODataApi/odata/7052_95/Perioden'}

In [18]:
url = urls['Geslacht']
response = requests.get(url)
data = response.json()
df_geslacht = pd.json_normalize(data, 'value')[['Key', 'Title']].set_index('Key')

In [24]:
df_geslacht

,Title
Key,
T001038,Totaal mannen en vrouwen
3000,Mannen
4000,Vrouwen


In [26]:
def json_to_dataframe(url, base = 'value', cols = ('Key', 'Title'), index_col='Key'):
    response = requests.get(url)
    data = response.json()
    df = pd.json_normalize(data, base)
    return df[list(cols)].set_index(index_col)

In [27]:
df_perioden = json_to_dataframe(urls['Perioden'])
df_perioden

,Title
Key,
1950JJ00,1950
1951JJ00,1951
1952JJ00,1952
1953JJ00,1953
1954JJ00,1954
...,...
2014JJ00,2014
2015JJ00,2015
2016JJ00,2016


In [34]:
df_geslacht = json_to_dataframe(urls['Geslacht'])
df_geslacht

,Title
Key,
T001038,Totaal mannen en vrouwen
3000,Mannen
4000,Vrouwen


In [30]:
csvfile='7052_95_UntypedDataSet_03052020_113458.csv'

In [42]:
df=pd.read_csv(csvfile, sep=';', index_col=0)
# df['Geslacht'] = df['Geslacht'].astype('string')
df.dtypes

Geslacht                                    int64
Leeftijd                                    int64
Perioden                                   object
TotaalOnderliggendeDoodsoorzaken_1          int64
TotaalInfectieuzeEnParasitaireZktn_2        int64
                                           ...   
k_172Zelfdoding_89                          int64
k_173MoordEnDoodslag_90                     int64
k_174GebeurtenissenOpzetOnbekend_91       float64
k_175OverigeUitwendigeDoodsoorzaken_92      int64
GemiddeldeBevolking_93                      int64
Length: 96, dtype: object

In [36]:
df['Geslacht'].map(df_geslacht['Title'])

ID
1587    NaN
1588    NaN
1589    NaN
1590    NaN
1591    NaN
       ... 
4480    NaN
4481    NaN
4482    NaN
4483    NaN
4484    NaN
Name: Geslacht, Length: 2760, dtype: object

In [46]:
df_geslacht

,Title
Key,
T001038,Totaal mannen en vrouwen
3000,Mannen
4000,Vrouwen


In [38]:
df.join(df_geslacht, on='Geslacht')

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat